In [1]:
from datetime import datetime
from portfolio.equity_portfolio import EquityPortfolio
from analyzer.analyzer import Analyzer as a
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
start = datetime(2018,1,1)
end = datetime(2021,1,1)

In [3]:
ep = EquityPortfolio(start,end,"test_portfolio")

In [8]:
ep.load()

100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:29<00:00, 14.89s/it]


In [9]:
ep.sim()

 13%|██████████▎                                                                   | 66/501 [00:14<01:38,  4.42it/s]


 27%|████████████████████▌                                                        | 134/501 [00:30<01:23,  4.40it/s]


 40%|███████████████████████████████                                              | 202/501 [00:45<01:04,  4.61it/s]


 54%|█████████████████████████████████████████▍                                   | 270/501 [01:00<00:51,  4.48it/s]


 67%|███████████████████████████████████████████████████▉                         | 338/501 [01:16<00:38,  4.29it/s]


 81%|██████████████████████████████████████████████████████████████▍              | 406/501 [01:31<00:21,  4.41it/s]


 95%|████████████████████████████████████████████████████████████████████████▊    | 474/501 [01:47<00:06,  4.39it/s]


100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [02:09<00:00, 65.00s/it]


In [10]:
trades = ep.backtest()

 13%|██████████▎                                                                   | 66/501 [00:15<01:41,  4.30it/s]


  0%|                                                                                         | 0/2 [00:28<?, ?it/s]


KeyboardInterrupt: 

In [12]:
ep

dict_keys(['rolling_percent', 'progress_report'])

In [ ]:
pva = a.pv_analysis(ep)
iva = a.industry_analysis(ep)

In [25]:
whut.connect()
check = whut.retrieve_sim(e)
whut.disconnect()

In [28]:
ep.strats["rolling_percent"]["class"].simmed

False